# Episode 5: Deutsch's Algorithm
Deutsch's Algorithm is one of the first algorithm to demonstrate quantum superiority against classical paradigm. By the way, It's still very theoretical and still did not provide significant change in term of practicality.


### The Problem:
Given there is four kind of function for 1 bit.

| $function$ | $result$ |
| :----: | :----: |
| $$f_1: Identity$$ | $$\ket{0} \rightarrow \ket{0}$$ $$\ket{1} \rightarrow \ket{1}$$ |
| $$f_2: Negation$$ | $$\ket{0} \rightarrow \ket{1}$$ $$\ket{1} \rightarrow \ket{0}$$ |
| $$f_3: Constant-0$$ | $$\ket{0} \rightarrow \ket{0}$$ $$\ket{1} \rightarrow \ket{0}$$ |
| $$f_4: Constant-1$$ | $$\ket{0} \rightarrow \ket{1}$$ $$\ket{1} \rightarrow \ket{1}$$ |

We define those that can possibly return as 0 or 1 depend on input as **balance**, and those independent from its input as **constant**. In this case; $f_1$ and $f_2$ are balance function, and $f_3$ and $f_4$ are constant function.

Now, we interested in solving whether the function is **balance** or **constant**. How???

### Solution:

In conclusion, we can simply define problem as followed:

**Deutsch's Problem In/Out**
> **Task:** Is it balance or constant?<br>
> **Input:** $f$ is function; $f \in$ {$f_1$, $f_2$, $f_3$, $f_4$}<br>
> **Output:** $Balance$ or $Constant$

### Approach 1: Classical - $O(2)$

In [2]:
# list all function 
def classical_function(case: int):
    if case not in [1, 2, 3, 4]:
        raise ValueError("`case` must be in set of {1,2,3,4}")
    def f_1(x: bool):
        return x
    def f_2(x: bool):
        return not x
    def f_3(x: bool):
        return False
    def f_4(x: bool):
        return True
    if case==1:
        return f_1
    if case==2:
        return f_2
    if case==3:
        return f_3
    if case==4:
        return f_4

This is the classical approach, simply input `True` and `False`, and determined whether is it equal or not

In [3]:
def solve(f):
    if f(True)==f(False):
        return "Constant"
    else:
        return "Balance"
display(solve(classical_function(1)))
display(solve(classical_function(2)))
display(solve(classical_function(3)))
display(solve(classical_function(4)))

'Balance'

'Balance'

'Constant'

'Constant'

### Approach 2: Quantum (Deutsch's Algorithm) - $O(1)$

Eventhough, the code is simple. Quantum algorithm can solve this in only one action with **Deutsch's algorithm**.

By the way, we have to encode function with quantum circuit.

In [4]:
from qiskit import QuantumCircuit

def quantum_function(case: int):
    if case not in [1, 2, 3, 4]:
        raise ValueError("`case` must be in set of {1,2,3,4}")
    f = QuantumCircuit(2)
    if case in [1, 2]:
        f.cx(0, 1)
    if case in [2, 4]:
        f.x(1)
    return f

In [5]:
# Here are list of encoded function in QuantumCircuit
display("f_1: identity", quantum_function(1).draw())
display("f_2: negative", quantum_function(2).draw())
display("f_3: constant-0", quantum_function(3).draw())
display("f_4: constant-1", quantum_function(4).draw())

'f_1: identity'

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

'f_2: negative'

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

'f_3: constant-0'

q_0: 
     
q_1:

'f_4: constant-1'

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

`q0` is used as input bit, while `q1` is used as auxilary bit that always input as $\ket{0}$.

Now we can solve them by following code:

In [ ]:
from qiskit_aer import AerSimulator

def deutsch_algorithm(f: QuantumCircuit):
    n = f.num_qubits-1
    qc = QuantumCircuit(n+1, n)
    
    qc.x(n)
    qc.h(range(n+1))

    qc.barrier()
    qc.compose(f, inplace=True)
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n),range(n))

    res = AerSimulator().run(qc,shot=1,memory=True).result()
    measurement = res.get_memory()
    if measurement[0]=="1":
        return qc,"Balance"
    return qc,"Constant"

In [12]:
n=int(input())

f = quantum_function(n)
display("Initial function: ", f.draw())
circ,ans = deutsch_algorithm(f)
display("Solver Circuit: ",circ.draw())
display(ans)


'Initial function: '

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

'Solver Circuit: '

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

'Balance'

### Mathematical Proof:
Comming Soon